# Basic routing

In this section we build a simple RESTful API application that use JSON to communicate using different HTTP methods. Before that we need to learn how to define a routing map. The easiest way and most popular of adding new routing map entries can be done with the ``@app.route`` annotation.

In [6]:
from flask import request
from flask import Flask

app = Flask(__name__)

@app.route('/')
def index():
    user_agent = request.headers.get('User-Agent')
    return '<p>Your browser is {}</p>'.format(user_agent)

if __name__ == '__main__':
    app.run(host="0.0.0.0")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.1 - - [02/Aug/2019 19:37:36] "GET / HTTP/1.1" 200 -
172.17.0.1 - - [02/Aug/2019 19:37:38] "GET /favicon.ico HTTP/1.1" 404 -


You can also add new routing map entries by using the ``add_url_rule`` method:

In [7]:
from flask import Flask

app = Flask(__name__)

def index():
    return '<h1>Hello World!</h1>'

app.add_url_rule('/', 'index', index)

if __name__ == '__main__':
    app.run(host="0.0.0.0")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.1 - - [02/Aug/2019 19:43:07] "GET / HTTP/1.1" 200 -


To build a RESTful application, we also need to specify the methods that we want to use for a given request. You can do it using the ``methods`` parameter:

In [16]:
from flask import request
from flask import Flask

app = Flask(__name__)

@app.route('/',methods=['GET'])
def hello_world():
    user_agent = request.headers.get('User-Agent')
    return 'GET'

@app.route('/hello',methods=['POST'])
def hello_post():
    user_agent = request.headers.get('User-Agent')
    return 'POST'

if __name__ == '__main__':
    app.run(host="0.0.0.0")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.1 - - [03/Aug/2019 15:11:53] "GET / HTTP/1.1" 200 -
172.17.0.1 - - [03/Aug/2019 15:12:05] "POST /hello HTTP/1.1" 200 -


You can test the two urls using:
- GET - the browser
- POST - curl or postman (https://www.getpostman.com/).

Keep in mind of the port mapping if used outside of the docker container. the curl example for POST request looks like below. You can easily change the url and the HTTP method into ``GET``.

In [ ]:
!curl -X POST http://localhost:5000/hello

A different way of running the flask application is to set the main application python file in the ``FLASK_APP`` variable and run it as a few cells below.

In [28]:
%env FLASK_APP=flask_sample.py

env: FLASK_APP="flask_routing1.py"


For Unix systems, you should also export environment variables to execute Python with UTF-8 encoding:

In [25]:
%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8

env: LC_ALL=C.UTF-8
env: LANG=C.UTF-8


You can also export the variables from console using the export command or set in Windows.

In [29]:
%%writefile flaskrouting1.py
from flask import request
from flask import Flask

app = Flask(__name__)

@app.route('/',methods=['GET'])
def hello_world():
    user_agent = request.headers.get('User-Agent')
    return 'GET'

@app.route('/hello',methods=['POST'])
def hello_post():
    user_agent = request.headers.get('User-Agent')
    return 'POST'

Writing flaskrouting1.py


Flask be run as following:

In [32]:
%env FLASK_APP=flaskrouting1.py
!flask run

env: FLASK_APP=flaskrouting1.py
 * Serving Flask app "flaskrouting1.py"
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
^C


# Dynamic routing

Dynamic routing is when you are able to send some information to the method that handles a specific route request. Below you can find an example of two functions where one gets one argument.

In [ ]:
from flask import Flask
app = Flask(__name__)

@app.route('/')
def index():
    return 'Hello World!'

@app.route('/grant/<title>')
def grant(title):
    return '<h1>Grant title: {}!</h1>'.format(title)

We can also set a specific type of the expected parameter like an ID that in many cases should be an integer:

In [ ]:
from flask import Flask
app = Flask(__name__)

@app.route('/')
def index():
    return 'Hello World!'

@app.route('/grant/<int:grant_id>')
def grant(grant_id):
    return '<h1>Grant id: {}!</h1>'.format(grant_id)

# Context

The application and request context is an important part of each flask application. Below you can find two application context variables and two request context variables. The first two are used to get the instance and object of the application.

| variable  | Description |
|---|---|
| current_app  | Application context. The application instance for the active application |
| g  | An object that the application can use for temporary storage during the handling of a request. This variable is reset with each request  |
| request  | The request object, which encapsulates the contents of an HTTP request sent by the client  |
| session  | The user session, a dictionary that the application can use to store values that are “remembered” between requests  |


In [10]:
from flask import current_app

app = Flask("Test")

current_app.name # an error is returned here, don't panic ;)

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.

To get the current application name we need to push the current application context first:

In [11]:
app_ctx = app.app_context()
app_ctx.push()
current_app.name

The ``g`` is a namespace object that is available everywhere in our application. You can think about it as a global variable. In the example below we initialize the database connection whenever ``get_db`` function is invoked and the ``_database`` variable isn't initialized. The annotation ``@app.teardown_appcontext`` sets a function that is invoked when the context is closing.

In [ ]:
from flask import g
import sqlite3

DATABASE = 'grants.db'

def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        db = g._database = sqlite3.connect(DATABASE)
    return db

@app.teardown_appcontext
def close_connection(exception):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

# Routing map

To check the whole routing map for our application, you can just use the ``url_map`` object:

In [13]:
app.url_map

Map([<Rule '/static/<filename>' (GET, HEAD, OPTIONS) -> static>])

# Request Object

The ``request`` object mentioned before will be used very often as it contain all information that we get from the user/browser when requesting our web application. There are three methods that are worth to mention:

| method  | description  |
|---|---|
| get_data  | Returns the buffered data from the request body  |
| get_json  | Returns a Python dictionary with the parsed JSON included in the body of the request  |
| is_secure()  | Returns True if the request came through a secure (HTTPS) connection  |

We have also many arguments that contain information on the request:

| argument  | description  |
|---|---|
| form  | A dictionary with all the form fields submitted with the request  |
| args  | A dictionary with all the arguments passed in the query string of the URL  |
| values  | A dictionary that combines the values in form and args  |
| cookies  | A dictionary with all the cookies included in the request  |
| headers  | A dictionary with all the HTTP headers included in the request  |
| files  | A dictionary with all the file uploads included with the request  |
| blueprint  | The name of the Flask blueprint that is handling the reques  |
| endpoint  | The name of the Flask endpoint that is handling the request. Flask uses the name of the view function as the endpoint name for a route  |
| method  | The HTTP request method, such as GET or POST  |
| scheme  | The URL scheme (http or https)  |
| host  | The host defined in the request, including the port number if given by the client  |
| path  | The path portion of the URL  |
| query_string  | The query string portion of the URL, as a raw binary value  |
| full_path  | The path and query string portions of the URL  |
| url  | The complete URL requested by the client  |
| base_url | Same as url, but without the query string component  |
| remote_addr  | The IP address of the clien  |
| environ  | The raw WSGI environment dictionary for the request  |

An example of getting the url given by the user is presented below:

In [14]:
from flask import Flask
from flask import request

app = Flask('Hello')

@app.route('/')
def show_url():
    return str(request.base_url)

if __name__ == '__main__':
    app.run(host="0.0.0.0")

 * Serving Flask app "Hello" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.1 - - [02/Aug/2019 22:20:37] "GET / HTTP/1.1" 200 -


### Response

We use many of the request arguments later in this and next notebooks. Most of the requests return also a response. There are four methods that the response object has:

| method  | description  |
|---|---|
| set_cookie  | Adds a cookie to the response  |
| delete_cookie  | Removes a cookie  |
| set_data  | Sets the response body as a string or bytes value  |
| get_data  | Gets the response body  |

An example of setting up the cookie is given below. We use the ``make_response`` function to create a response object.

In [ ]:
from flask import make_response

@app.route('/')
def index():
    response = make_response('Hello World!')
    response.set_cookie('grant_id', '102')
    return response

We have also four arguments of the response object:

| argument  | description  |
|---|---|
| status_code  | The numeric HTTP status code  |
| headers  | A dictionary-like object with all the headers that will be sent with the response  |
| content_length  | The length of the response body  |
| content_type  | The media type of the response body  |


We can set the status code of the response like 200, 404 and so on. We can set the headers or content type to application/json or other type.

#### Exercise 1. Create a REST API routing map for grants

Just to point out a few features:

- the application should accept POST data to add to the ``grants`` variable,
- set routes to get the data about grants by ``id`` via ``GET`` method,
- set routes to get the list of grants by ``id`` via ``GET`` method,
- set routes to delete a grant by ``id`` via ``DELETE`` method.

Use postman or curl to check if the application fullfil the requirements.

In [ ]:
!curl -d "param1=value1&param2=value2" -X POST http://localhost:500/<route>

In [ ]:
# your code goes here

# Preparing and parsing JSON

A RESTful application uses JSON to communicate. It uses also vary of HTTP methods, where four are the most important: ``GET``, ``POST``, ``DELETE``, and ``PUT``. Python provides the ``json`` package to encode and build JSON data. An example of building JSON format responses is presented below.

In [36]:
import json

grantDict1 = {
  'id': 1,
  'title': 'Sample grant',
  'description': 'Very important one',
  'value': 10000
}
json_response = json.dumps(grantDict1)
print(json_response)

{"id": 1, "title": "Sample grant", "description": "Very important one", "value": 10000}


Using the ``dumps`` method we can easily convert almost any type to JSON. Below a list of dictionaries is converted into JSON.

In [38]:
grantDict2 = {
  'id': 2,
  'title': 'Second sample grant',
  'description': 'Even moreimportant one',
  'value': 200000
}

list_of_grants = []
list_of_grants.append(grantDict1)
list_of_grants.append(grantDict2)

json_response = json.dumps(list_of_grants)
print(json_response)
print(type(json_response))

[{"id": 1, "title": "Sample grant", "description": "Very important one", "value": 10000}, {"id": 2, "title": "Second sample grant", "description": "Even moreimportant one", "value": 200000}]
<class 'str'>


The ``loads`` method can be used to convert a string into objects that correspont to the JSON and Python as in the list

| JSON  | Python  |
|---|---|
| object  |  dict |
| array  | list  |
| string  | str  |
| number (int)  |  int |
| number (real)  | float  |
| true  | True  |
| false  | False  |
| null  | None  |

In [40]:
decoded = json.loads(json_response)
print(decoded)
print(type(decoded))

[{'id': 1, 'title': 'Sample grant', 'description': 'Very important one', 'value': 10000}, {'id': 2, 'title': 'Second sample grant', 'description': 'Even moreimportant one', 'value': 200000}]
<class 'list'>


#### Exercise 2. Convert previous exercise and add JSON responses and requests

The goal is to response with a JSON to any ``GET`` request. Interpret a JSON ``POST`` request and interpret the payload. Please keep in mind to set the content type for each ``GET`` and ``POST`` request. It should be set to ``application/json``. Each ``POST`` should return with an error_code response:

- 202 accepted,
- 403 forbidden.

In [ ]:
# your code goes here

# Upload files with Flask-Uploads




In [ ]:
from flask import Flask
from flask import request
from flask_uploads import UploadSet, configure_uploads, ALL, patch_request_class

app = Flask('Upload example')
app.config['UPLOADS_DEFAULT_DEST'] = '/home/codete/workshop/'

pdfs = UploadSet('files', ALL)
configure_uploads(app, pdfs)
patch_request_class(app)  # set maximum file size, default is 16MB

@app.route('/upload', methods=['POST'])
def upload_file():
    filename = pdfs.save(request.files['file'])
    file_url = pdfs.url(filename)
    return "Thank you: " + str(file_url)

if __name__ == '__main__':
    app.run(host="0.0.0.0")

#### Exercise 3. Handle the upload files for images

Change the code to accept only image files. You can use curl to test your solution:

In [ ]:
!curl -F 'file=@<path>/<file>' -X POST http://127.0.0.1:5000/upload

In [ ]:
# your code goes here

# Error handling (optional)

We can set a customized page for HTTP error codes like 404. We can use for it the annotation ``@app.errorhandler`` as presented below.

In [ ]:
from flask import Flask
from flask import request
app = Flask(__name__)

@app.route('/')
def hello_world():
    return str(request.base_url)

@app.errorhandler(404)
def page_not_found(e):
    return "Not working :("

if __name__ == '__main__':
    app.run(host="0.0.0.0")

# Request hooks (optional)

We can also setup request hooks to execute a function before or after a request.

| annotation   | description  |
|---|---|
| before_first_request  | This annotation registers a function to execute before the first request is handled  |
| before_request  | This annotation registers a function to execute before a request is handled  |
| after_request  | This annotation registers a function to execute after a request is handled. The registered function will not be called in case an unhandled exception occurred in the request handler. The function must accept a response object and return the same or new response  |
| teardown_request  |  Similar to after_request annotation but the registered function will always execute regardless of whether the request handler throws an exception or not |

An example of ``@app.before_first_request`` is given below.

In [ ]:
@app.before_first_request
def sample():
    print("ok")